In [1]:
import requests

URL = "https://s199-de.ogame.gameforge.com/api/universe.xml"

response = requests.get(URL)
with open('universe.xml', 'wb') as file:
    file.write(response.content)

    

In [2]:
import csv 
import xml.etree.ElementTree as ET 

tree = ET.parse('universe.xml')

In [3]:
root = tree.getroot()

In [4]:
open_coords = [f"{g}:{s}:8" for g in range(1, 9) for s in range(1, 500) ]
for planet in root.findall('planet'):
    coords = planet.get("coords")
    if coords[-1] == "8":
        open_coords.remove(coords)
open_coords

['8:387:8']

In [5]:
timestamp = int(root.get('timestamp'))
timestamp

1718104434

In [6]:
from datetime import datetime
print(datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'))


2024-06-11 11:13:54
